# Exploring Hackers News Posts

In this project, we'll work with a data set of submissions to popular technology site Hacker News.

<img src= https://s3.amazonaws.com/dq-content/354/hacker_news.jpg />

Hacker News is a site started by the startup incubator Y Combinator, where user-submitted stories (known as "posts") are voted and commented upon, similar to reddit. Hacker News is extremely popular in technology and startup circles, and posts that make it to the top of Hacker News' listings can get hundreds of thousands of visitors as a result.

We're specifically interested in posts whose titles begin with either Ask HN or Show HN. Users submit Ask HN posts to ask the Hacker News community a specific question. Likewise, users submit Show HN posts to show the Hacker News community a project, product, or just generally something interesting.

We'll compare these two types of posts to determine the following:

Do Ask HN or Show HN receive more comments on average?
Do posts created at a certain time receive more comments on average?

## Introduction

You can find the data set here, but note that it has been reduced from almost 300,000 rows to approximately 20,000 rows by removing all submissions that did not receive any comments, and then randomly sampling from the remaining submissions. Below are descriptions of the columns:

* **id**: The unique identifier from Hacker News for the post
* **title**: The title of the post
* **url**: The URL that the posts links to, if it the post has a URL
* **num_points**: The number of points the post acquired, calculated as the total number of upvotes minus the total number of downvotes
* **num_comments**: The number of comments that were made on the post
* **author**: The username of the person who submitted the post
* **created_at**: The date and time at which the post was submitted



In [22]:
from csv import reader

hacker_news = list(reader(open('hacker_news.csv')))
hacker_news[:5]

[['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at'],
 ['12224879',
  'Interactive Dynamic Video',
  'http://www.interactivedynamicvideo.com/',
  '386',
  '52',
  'ne0phyte',
  '8/4/2016 11:52'],
 ['10975351',
  'How to Use Open Source and Shut the Fuck Up at the Same Time',
  'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/',
  '39',
  '10',
  'josep2',
  '1/26/2016 19:30'],
 ['11964716',
  "Florida DJs May Face Felony for April Fools' Water Joke",
  'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/',
  '2',
  '1',
  'vezycash',
  '6/23/2016 22:20'],
 ['11919867',
  'Technology ventures: From Idea to Enterprise',
  'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429',
  '3',
  '1',
  'hswarna',
  '6/17/2016 0:01']]

## Removing Headers from a List of Lists

In [24]:
headers = hacker_news[0]
headers

['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']

In [26]:
hn = hacker_news[1:]
hn[:5]

[['12224879',
  'Interactive Dynamic Video',
  'http://www.interactivedynamicvideo.com/',
  '386',
  '52',
  'ne0phyte',
  '8/4/2016 11:52'],
 ['10975351',
  'How to Use Open Source and Shut the Fuck Up at the Same Time',
  'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/',
  '39',
  '10',
  'josep2',
  '1/26/2016 19:30'],
 ['11964716',
  "Florida DJs May Face Felony for April Fools' Water Joke",
  'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/',
  '2',
  '1',
  'vezycash',
  '6/23/2016 22:20'],
 ['11919867',
  'Technology ventures: From Idea to Enterprise',
  'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429',
  '3',
  '1',
  'hswarna',
  '6/17/2016 0:01'],
 ['10301696',
  'Note by Note: The Making of Steinway L1037 (2007)',
  'http://www.nytimes.com/2007/11/07/movies/07stein.html?_r=0',
  '8',
  '2',
  'walterbell',
  '9/30/2015 4:12']]

## Extracting Ask HN and Show HN Posts
Since we're only concerned with post titles beginning with Ask HN or Show HN, we'll create new lists of lists containing just the data for those titles.

In [29]:
# Separate posts beginning with Ask HN and Show HN (and case variations) into two different lists next.
ask_posts = []
show_posts = []
other_posts = []

for post in hn:
    if post[1].lower().startswith('ask hn'):
        ask_posts.append(post)
    elif post[1].lower().startswith('show hn'):
        show_posts.append(post)
    else:
        other_posts.append(post)
        
print(len(ask_posts))
print(len(show_posts))
print(len(other_posts))
    

1744
1162
17194


## Calculating the Average Number of Comments for Ask HN and Show HN Posts

Now that we separated ask posts and show posts into different lists, we'll calculate the average number of comments each type of post receives.

In [30]:
# Calculate the average number of comments `Ask HN` posts receive.
total_ask_com = 0
for post in ask_posts:
    total_ask_com += int(post[4])

avg_ask_com = total_ask_com / len(ask_posts)
avg_ask_com

14.038417431192661

In [32]:
total_show_com = 0
for post in show_posts:
    total_show_com += int(post[4])

avg_show_com = total_show_com / len(show_posts)
avg_show_com

10.31669535283993

Thus, we can see that ask posts exceeds show posts by the average amount of comment by almost 50%. Accordingly, the audience engagement of ask posts is higher, so we'll focus our remaining analysis just on these posts.

## Finding the Amount of Ask Posts and Comments by Hour Created

Next, we'll determine if ask posts created at a certain time are more likely to attract comments.

In [54]:
#  Calculate the amount of ask posts created during each hour of day and the number of comments received.
import datetime as dt
result = []
for post in ask_posts:
    result.append([dt.datetime.strptime(post[6], '%m/%d/%Y %H:%M'),
                   int(post[4])])

counts_by_hour = {}
comments_by_hour = {}
for each in result:
    hour = each[0].strftime('%H')
    if hour in counts_by_hour:
        comments_by_hour[hour] += each[1]
        counts_by_hour[hour] += 1
    else:
        comments_by_hour[hour] = each[1]
        counts_by_hour[hour] = 1

comments_by_hour

{'00': 447,
 '01': 683,
 '02': 1381,
 '03': 421,
 '04': 337,
 '05': 464,
 '06': 397,
 '07': 267,
 '08': 492,
 '09': 251,
 '10': 793,
 '11': 641,
 '12': 687,
 '13': 1253,
 '14': 1416,
 '15': 4477,
 '16': 1814,
 '17': 1146,
 '18': 1439,
 '19': 1188,
 '20': 1722,
 '21': 1745,
 '22': 479,
 '23': 543}

## Calculating the Average Number of Comments for Ask HN Posts by Hour

In [58]:
# create a list of lists containing the hours during which posts were created and the average number of comments those posts received
avg_by_hour = []
for hour in comments_by_hour:
    avg_by_hour.append([hour, (comments_by_hour[hour] / counts_by_hour[hour])])
avg_by_hour

[['18', 13.20183486238532],
 ['01', 11.383333333333333],
 ['16', 16.796296296296298],
 ['22', 6.746478873239437],
 ['17', 11.46],
 ['13', 14.741176470588234],
 ['12', 9.41095890410959],
 ['15', 38.5948275862069],
 ['07', 7.852941176470588],
 ['04', 7.170212765957447],
 ['11', 11.051724137931034],
 ['05', 10.08695652173913],
 ['03', 7.796296296296297],
 ['19', 10.8],
 ['10', 13.440677966101696],
 ['21', 16.009174311926607],
 ['00', 8.127272727272727],
 ['09', 5.5777777777777775],
 ['08', 10.25],
 ['02', 23.810344827586206],
 ['06', 9.022727272727273],
 ['14', 13.233644859813085],
 ['23', 7.985294117647059],
 ['20', 21.525]]

## Sorting and Printing Values from a List of Lists

In [62]:
swap_avg_by_hour = []
for each in avg_by_hour:
    swap_avg_by_hour.append([each[1], each[0]])
sorted_swap = sorted(swap_avg_by_hour, reverse=True)
sorted_swap

[[38.5948275862069, '15'],
 [23.810344827586206, '02'],
 [21.525, '20'],
 [16.796296296296298, '16'],
 [16.009174311926607, '21'],
 [14.741176470588234, '13'],
 [13.440677966101696, '10'],
 [13.233644859813085, '14'],
 [13.20183486238532, '18'],
 [11.46, '17'],
 [11.383333333333333, '01'],
 [11.051724137931034, '11'],
 [10.8, '19'],
 [10.25, '08'],
 [10.08695652173913, '05'],
 [9.41095890410959, '12'],
 [9.022727272727273, '06'],
 [8.127272727272727, '00'],
 [7.985294117647059, '23'],
 [7.852941176470588, '07'],
 [7.796296296296297, '03'],
 [7.170212765957447, '04'],
 [6.746478873239437, '22'],
 [5.5777777777777775, '09']]

In [75]:
# Find Top 5 Hours for Ask Posts Comments
print("Top 5 Hours for Ask Posts Comments")
for each in sorted_swap[:5]:
    print('At {hour} o\'clock: {avg:.2f} average comments per post'
          .format(hour=dt.datetime.strptime(each[1],'%H').hour, avg=each[0]))

Top 5 Hours for Ask Posts Comments
At 15 o'clock: 38.59 average comments per post
At 2 o'clock: 23.81 average comments per post
At 20 o'clock: 21.52 average comments per post
At 16 o'clock: 16.80 average comments per post
At 21 o'clock: 16.01 average comments per post


From the results it is clear that the most comments were received by posts published at 15 o’clock. The average value of comments in posts at this time significantly exceeds the value of the second-rated element (2:00). For the correct conclusion should take into account the timezone is Eastern Time in the US.

## Conclusion

In this project, we analyzed the `'ask posts'` and `'show posts'` to determine which type of post and time receive the most comments on average. Based on our analysis, in order to maximize the number of comments a post receives, we recommend classifying the post as a post request and creating it from 15:00 to 16:00 (15:00 - 16:00 Eastern time).
